In [ ]:
%pip install --upgrade --quiet  langchain langchain-community langchain-openai langchain-experimental neo4j wikipedia tiktoken yfiles_jupyter_graphs

In [2]:
from langchain_core.runnables import (
    RunnableBranch,
    RunnableLambda,
    RunnableParallel,
    RunnablePassthrough,
)

In [3]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.prompts.prompt import PromptTemplate

In [ ]:
OPENAI_API_KEY=""

In [5]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.prompts.prompt import PromptTemplate
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain_core.messages import AIMessage, HumanMessage
from langchain_core.output_parsers import StrOutputParser
from langchain_community.graphs import Neo4jGraph
from langchain_community.vectorstores import Neo4jVector
from langchain_experimental.graph_transformers import LLMGraphTransformer
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain.text_splitter import TokenTextSplitter
from langchain.docstore.document import Document
from typing import Tuple, List, Optional
import os

/home/anand/.local/lib/python3.10/site-packages/IPython/core/interactiveshell.py:3579: LangChainDeprecationWarning: As of langchain-core 0.3.0, LangChain uses pydantic v2 internally. The langchain_core.pydantic_v1 module was a compatibility shim for pydantic v1, and should no longer be used. Please update the code to import from Pydantic directly.

For example, replace imports like: `from langchain_core.pydantic_v1 import BaseModel`
with: `from pydantic import BaseModel`
or the v1 compatibility namespace if you are working in a code base that has not been fully upgraded to pydantic 2 yet. 	from pydantic.v1 import BaseModel

  exec(code_obj, self.user_global_ns, self.user_ns)


In [6]:
NEO4J_URI="neo4j+s://71d8f74b.databases.neo4j.io"
NEO4J_USERNAME="neo4j"
NEO4J_PASSWORD="4ZuShP-Ats28MznYfC9kjCW5nShc6FT6iSho69Z9FVU"


In [7]:
os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY
os.environ["NEO4J_URI"] = NEO4J_URI
os.environ["NEO4J_USERNAME"] = NEO4J_USERNAME
os.environ["NEO4J_PASSWORD"] = NEO4J_PASSWORD

In [9]:
# Connect to Neo4j
graph = Neo4jGraph()

In [10]:
import os
import tempfile
import re
import PyPDF2
import docx2txt

# Assuming you have these dependencies installed
# If using spaCy (mentioned in extract_entities function):
# import spacy
# nlp = spacy.load("en_core_web_sm")
# stop_words = nlp.Defaults.stop_words

# Text extraction functions from your original code
def extract_text_from_file(file):
    """
    Extract text content from various file formats.
    
    Args:
        file: Uploaded file object
    
    Returns:
        str: Extracted text
    """
    file_name = file.name.lower()
    file_content = file.read()
    
    # Create a temporary file to store content
    with tempfile.NamedTemporaryFile(delete=False, suffix=os.path.splitext(file_name)[1]) as tmp_file:
        tmp_file.write(file_content)
        tmp_file_path = tmp_file.name
    
    try:
        # Extract text based on file type
        if file_name.endswith('.pdf'):
            text = extract_from_pdf(tmp_file_path)
        elif file_name.endswith('.docx'):
            text = extract_from_docx(tmp_file_path)
        elif file_name.endswith('.txt'):
            try:
                with open(tmp_file_path, 'r', encoding='utf-8', errors='ignore') as f:
                    text = f.read()
            except Exception as e:
                # Fallback for text files
                text = file_content.decode('utf-8', errors='ignore')
        else:
            text = ""
            raise ValueError(f"Unsupported file format: {os.path.splitext(file_name)[1]}")
    except Exception as e:
        # Log the error for debugging
        print(f"Error extracting text: {str(e)}")
        # Provide a generic error message
        text = f"Error processing file: {str(e)}"
    finally:
        # Clean up temporary file
        if os.path.exists(tmp_file_path):
            os.unlink(tmp_file_path)
    
    return text

def extract_from_pdf(file_path):
    """Extract text from PDF file"""
    text = ""
    try:
        with open(file_path, 'rb') as file:
            pdf_reader = PyPDF2.PdfReader(file)
            for page_num in range(len(pdf_reader.pages)):
                page = pdf_reader.pages[page_num]
                text += page.extract_text() + "\n"
    except Exception as e:
        text = f"Error extracting PDF content: {str(e)}"
    return text

def extract_from_docx(file_path):
    """Extract text from DOCX file"""
    try:
        text = docx2txt.process(file_path)
        return text
    except Exception as e:
        return f"Error extracting DOCX content: {str(e)}"

In [11]:
def create_text_chunks(text, window_size=50000, overlap=10000):
    """
    Split large text into overlapping chunks using a sliding window approach.
    Attempts to break at natural boundaries like paragraphs or sentences.
    
    Args:
        text: Input text to be chunked
        window_size: Maximum size of each chunk in characters
        overlap: Overlap between consecutive chunks in characters
    
    Returns:
        list: List of tuples containing (start_pos, end_pos, chunk_text)
    """
    chunks = []
    
    # Only chunk if text is larger than window size
    if len(text) <= window_size:
        return [(0, len(text), text)]
    
    # Create overlapping chunks with sliding window approach
    start_positions = range(0, len(text), window_size - overlap)
    
    for pos in start_positions:
        # Calculate the ideal end position
        ideal_end = min(pos + window_size, len(text))
        
        # If not at the end of text, try to find a natural boundary near the ideal end
        if ideal_end < len(text):
            # Look for paragraph break near the ideal end
            paragraph_break = text.rfind('\n\n', pos, ideal_end)
            if paragraph_break != -1 and paragraph_break > pos + (window_size // 2):
                # If paragraph break found and it's not too close to the start, use it
                end_pos = paragraph_break
            else:
                # Look for sentence break (period followed by space)
                sentence_break = text.rfind('. ', pos + (window_size // 2), ideal_end)
                if sentence_break != -1:
                    end_pos = sentence_break + 1  # Include the period
                else:
                    # No natural break found, use the ideal end
                    end_pos = ideal_end
        else:
            # At the end of text, use the remaining text
            end_pos = ideal_end
            
        chunks.append((pos, end_pos, text[pos:end_pos]))
        
        # Break if we've reached the end of the text
        if end_pos >= len(text):
            break
            
    return chunks

In [12]:
import os
import tempfile
import re
import PyPDF2
import docx2txt
from typing import List, Dict, Any, Optional

# LangChain imports
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import Chroma
from langchain.chains import RetrievalQA
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.schema.document import Document

In [13]:
# RAG Implementation using LangChain
class PDFRagSystem:
    def __init__(self, model_name="gpt-3.5-turbo", temperature=0, chunk_size=1000, chunk_overlap=200):
        """
        Initialize the RAG system for PDF documents.
        
        Args:
            model_name: Name of the LLM model to use
            temperature: Temperature setting for the LLM
            chunk_size: Size of text chunks for the document splitter
            chunk_overlap: Overlap between chunks
        """
        self.llm = ChatOpenAI(model_name=model_name, temperature=temperature)
        self.embeddings = OpenAIEmbeddings()
        self.text_splitter = RecursiveCharacterTextSplitter(
            chunk_size=chunk_size, 
            chunk_overlap=chunk_overlap,
            length_function=len
        )
        self.vectorstore = None
        self.retriever = None
        self.qa_chain = None
    
    def process_file(self, file):
        """
        Process a PDF file and create a vector database for retrieval.
        
        Args:
            file: File object for a PDF document
        
        Returns:
            self: For method chaining
        """
        # Extract text from the file
        print(f"Processing file: {file.name}")
        text = extract_text_from_file(file)
        
        # Create LangChain documents from the extracted text
        raw_documents = [Document(page_content=text, metadata={"source": file.name})]
        
        # Split documents into chunks
        print("Splitting document into chunks...")
        documents = self.text_splitter.split_documents(raw_documents)
        print(f"Created {len(documents)} document chunks")
        
        # Create vector store
        print("Creating vector store...")
        self.vectorstore = Chroma.from_documents(documents, self.embeddings)
        self.retriever = self.vectorstore.as_retriever(search_kwargs={"k": 4})
        
        # Create QA chain
        template = """
        You are an AI assistant that helps users answer questions based on the provided document content.
        Use the following pieces of context to answer the question at the end.
        If you don't know the answer, just say that you don't know, don't try to make up an answer.
        Use three sentences maximum and keep the answer concise.
        
        {context}
        
        Question: {question}
        """
        
        PROMPT = PromptTemplate(
            template=template,
            input_variables=["context", "question"]
        )
        
        self.qa_chain = RetrievalQA.from_chain_type(
            llm=self.llm,
            chain_type="stuff",
            retriever=self.retriever,
            return_source_documents=True,
            chain_type_kwargs={"prompt": PROMPT}
        )
        
        print("RAG system ready for queries!")
        return self
    
    def ask(self, question):
        """
        Ask a question to the RAG system.
        
        Args:
            question: Question to ask about the document
        
        Returns:
            dict: Answer and source documents
        """
        if not self.qa_chain:
            raise ValueError("No document has been processed. Call process_file() first.")
        
        print(f"Question: {question}")
        result = self.qa_chain({"query": question})
        
        # Format and return the result
        answer = result["result"]
        source_docs = result["source_documents"]
        
        print(f"Answer: {answer}")
        print("\nSources:")
        for i, doc in enumerate(source_docs):
            print(f"Source {i+1}: {doc.page_content[:150]}...")
        
        return {
            "answer": answer,
            "source_documents": source_docs
        }








In [ ]:
import io

# Read the file content
with open('Financial Crisis Enquiry Report 2008.pdf', 'rb') as f:
    file_content = f.read()

# Create a BytesIO object with the content
file_obj = io.BytesIO(file_content)
file_obj.name = 'Financial Crisis Enquiry Report 2008.pdf'  # This is now allowed

# Now use it with your RAG system
rag = PDFRagSystem()
rag.process_file(file_obj)


In [ ]:
print(rag.ask("Please identify the detault event. For the event identify the defaulting organization and the time when the default event occurred")["answer"])
